In [ ]:
import numpy as np
import pulser
import igraph

import matplotlib.pyplot as plt

from pulser import Pulse, Sequence, Register, Simulation
from pulser.devices import Chadoq2

from scipy.optimize import minimize

# Introduction 

In this tutorial, we illustrate how to solve the Maximum Independent Set (MIS) problem using the Quantum Approximate Optimization Algorithm procedure on a platform of Rydberg atoms in analog mode, using Pasqal's library Pulsor. 

For more details about this problem and how to encode it on a Rydberg atom quantum processor, see [Pichler, et al., https://arxiv.org/abs/1808.10816, Henriet, https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.012335 and Dalyac, et al., https://arxiv.org/abs/2012.14859].

Consider an undirected graph composed of a set of vertices connected by unweighted edges. An independent set of this graph is a subset of vertices where any two elements of this subset are not connected by an edge. The Maximum Independent Set (MIS) corresponds to the largest of such subsets, and it is in general an NP-complete problem to determine the MIS of a graph. 

The MIS problem has several interesting applications, such as portfolio diversification in
finance, or broadcast systems (wifi or cellular network) optimization.

For example, assume an ensemble of identical radio transmitters over French cities that each have the same radius of transmission. It was quickly realized that two transmitters with close or equal frequencies could interfere with one another, hence the necessity to assign non-interfering frequencies to overlapping transmiting towers. Because of the limited amount of bandwith space, some towers have to be assigned the same or close frequencies. The MIS of a graph of towers indicate the maximum number of towers that can have close or equal given frequency (red points). 

<img src="files/France_MIS.png" alt="MIS map France" style="width: 700px;"/>


Here, we will show how to solve the MIS problem on Unit-Disk (UD) graphs with an excellent approximation.

When looking for the MIS of a graph, we separate the nodes into two distinct classes: an independence one and the others. We can attribute a status $z$ to each node, where $z_i = 1$  if node $i$ is attributed to the independent set, and $z_i=0$ otherwise. The Maximum Independent Set corresponds to  the minima of the following cost function: 

\begin{equation}
   C(z_1,\dots,z_N) = -\sum_{i=1}^N z_i + U \sum_{\langle i,j \rangle}z_i z_j
 \label{cost_function}
\end{equation}
where $U \gg 1$  and $\langle i,j \rangle$ represents adjacent nodes (i.e. there is a link between node $i$ and $j$). In this cost function, we want to promote a maximal number of atoms to the $1$ state, but the fact that $U \gg 1$  strongly penalizes two adjacent vertices in state $1$. The minimum of $C(z_0,\dots,z_N)$ therefore corresponds to the maximum independent set of the graph.

Interestingly, the operator $\hat{C}$ associated with the cost function of the previous equation can be natively realized on a neutral atom platform with some constraints on the graph edges. We map a  ground state and a Rydberg state of each atom to a spin $1/2$, where $|0 \rangle = |r \rangle$ is a Rydberg state and $|1 \rangle = |g \rangle$ is a ground state. An atom in a Rydberg state has an excited electron with a very high principal quantum number and therefore exhibits a huge electric dipole moment. As such, when two atoms are excited to Rydberg states, they exhibit a strong van der Waals interaction. Placing $N$ atoms at positions $\textbf{r}_j$ in a 2D plane, and coupling the ground state $|1\rangle$ to the Rydberg state $|0\rangle$ with a laser system enables the realization of the Hamiltonian :
\begin{equation}
    H= \sum_{i=1}^N \frac{\hbar\Omega}{2} \sigma_i^x - \sum_{i=1}^N \frac{\hbar \delta}{2}  \sigma_i^z+\sum_{j<i}\frac{C_6}{|\textbf{r}_i-\textbf{r}_j|^{6}} n_i n_j.
\label{eq:ising_Hamiltonian}
\end{equation}
Here, $\Omega$ and $\delta$ are respectively the Rabi frequency and detuning of the laser system and $\hbar$ is the reduced Planck constant. The first two terms of the equation govern the transition between states $|0\rangle$ and $|1 \rangle$ induced by the laser, while the third term represents the repulsive Van der Waals interaction between atoms in the $|0\rangle$ state. More precisely, $n_i = (\sigma_i
^z + 1)/2$ counts the number of Rydberg excitations at position $i$. The interaction strength between two atoms decays as $|\textbf{r}_i-\textbf{r}_j|^{-6}$.

# From a graph to an atomic register

We now illustrate how one can use Pulser and a Pasqal device to find the MIS of a UD-graph. Because the quantum platform is emulated in this notebook, we restrict the number of atoms to 5, just to show a proof-of-concept.

In [ ]:
def pos_to_graph(pos, d = Chadoq2.rydberg_blockade_radius(1)): #d = rydberg blockade
    edges=[]
    for n in range(len(pos)-1):
        for m in range(n+1, len(pos)):
            pwd = ((pos[m][0]-pos[n][0])**2+(pos[m][1]-pos[n][1])**2)**0.5
            #print('distance=%f'%pwd)
            if pwd < d:
                edges.append([n,m])                         # Below a given distance, vertices are connected
    #Now we use a igraph object to describe our graph. This module has several interesting built-in functions.
    g=igraph.Graph()
    g.add_vertices(len(pos))
    g.add_edges(edges)
    return g

Here, we create an atomic register with 5 atoms and the corresponding graph.

In [ ]:
pos = np.array([[0., 0.], [-4, -7], [4,-7], [8,6], [-8,6]])
G = pos_to_graph(pos) 
qubits = dict(enumerate(pos))
reg = Register(qubits)
reg.draw()

In [ ]:
import networkx as nx
A  = np.matrix([[0, 1, 1, 1, 1], [1, 0, 1, 0, 0], [1, 1, 0, 0, 0], [1, 0, 0, 0, 0], [1, 0, 0, 0, 0]])
G_nx = nx.from_numpy_matrix(A)
fig,ax1 = plt.subplots(figsize = (4,4))
nx.draw(G_nx,
        pos,
        edge_color='blue', 
        ax=ax1,
        width=1,
        linewidths=1,
        node_size=250,
        node_color='g',
        alpha=1.,
        labels={node:node for node in G_nx.nodes()}
        )

This graph has two maximal independent sets: $(1,3,4)$ and $(2,3,4)$, respectively $'10100'$ and $'11000'$ in binary. 

# 2 Build the quantum loop 

Now, we must build the quantum part of the QAOA. All atoms are initially in the groundstate $|00\dots0\rangle$ of the `ground-rydberg`basis.  We then apply $p$ layers of alternating non-commutative Hamiltonians. The first one, called the mixing Hamiltonian $H_M$, is realized by taking $\Omega = 1$ MHz, and $\delta = 0 $ Mhz in the Hamiltonian equation. The second Hamiltonian $H_c$ is realized with $\Omega = \delta = 1$ MHz. $H_M$ and $H_c$ are applied turn in turn with parameters $\tau$ and $t$ respectively. A classical optimizer is then used to estimate the optimal parameters. 

Experimentally, we don't have access to the state vector $\psi$. We therefore make it more realistic by taking samples from the state vector that results from running the simulation with `simul.run()`. This is done with the built-in method `results.sample_final_state()`, in which we add the measurement basis which was declared at the end of the sequence, and the number of samples desired. Currently, the repetition rate of the machine is $5$Hz.

In [ ]:
def quantum_loop(param, r=reg):
    seq = Sequence(r, Chadoq2)
    seq.declare_channel('ch0','rydberg_global')
    middle = int(len(param)/2)
    param = np.array(param)*1 #wrapper 
    t = param[:middle] #associated to H_c
    tau = param[middle:] #associated to H_0
    p = len(t)
    for i in range(p):
        pulse_1 = Pulse.ConstantPulse(tau[i], 1., 0, 0) # H_M 
        pulse_2 = Pulse.ConstantPulse(t[i], 1., 1., 0) # H_M + H_c 
        seq.add(pulse_1, 'ch0')
        seq.add(pulse_2, 'ch0')
    seq.measure('ground-rydberg')
    simul = Simulation(seq, skip=100)
    results = simul.run()
    count_dict = results.sample_final_state(N_samples=1000) #sample from the state vector 
    return count_dict 

In [ ]:
example_dict = quantum_loop([300,900], r = reg)

We can then plot the distribution of the samples, to see the most frequent bitstrings sampled.

In [ ]:
def plot_distribution(C):
    C = dict(sorted(C.items(), key=lambda item: item[1], reverse=True))
    color_dict = {key: 'g' for key in C}
    indexes = ['01011', '00111']  # MIS indexes
    for i in indexes: 
        color_dict[i] = 'red'
    plt.figure(figsize=(12,6))
    plt.xlabel("bitstrings")
    plt.ylabel("counts")
    plt.bar(C.keys(), C.values(), width=0.5, color = color_dict.values())
    plt.xticks(rotation='vertical')
    plt.show()

In [ ]:
plot_distribution(example_dict)

The bitstrings '01011' and '00111' (in red) correspond to the two MIS of the graph. The goal of QAOA is to choregraph interferences between the basis states, in order to maximize the frequency of the MIS states. 

# Optimization 

We estimate the cost of a sampled state vector by making an average over the samples. Determining the cost of a given bitstring takes polynomial time. The average estimate is then used in the classical loop to optimize the variational parameters $\tau$ and $t$.

In [ ]:
def get_cost_colouring(z,G,penalty=10):
    """G: the graph (igraph)
       z: a binary colouring
       returns the cost of the colouring z, depending on the adjacency of the graph"""
    cost = 0 
    A = G.get_adjacency()
    z = np.array(tuple(z),dtype=int)
    for i in range(len(z)):
        for j in range(i,len(z)):
            cost += A[i][j]*z[i]*z[j]*penalty # if there's an edge between i,j and they are both in R state.
    
    cost -= np.sum(z) #to count for the 0s instead of the 1s 
    return cost 

def get_cost(counter,G):
    cost = 0 
    for key in counter.keys():
        cost_col = get_cost_colouring(key,G)
        cost += cost_col * counter[key]
    return cost / sum(counter.values())

In [ ]:
get_cost_colouring('00111',G)

In [ ]:
def func(param,*args):
    G = args[0]
    C = quantum_loop(param, r=reg)
    cost = get_cost(C,G)
    return cost


# QAOA for p = 1

`minimize` takes the as arguments `func`, the graph `G`and an initial `x0` point for the simplex.

In [ ]:
res = minimize(func, args=G, x0=np.array([1000,9000]),method='Nelder-Mead', tol=1e-5)

In [ ]:
count_dict = quantum_loop(res.x)
plot_distribution(count_dict)

QAOA is capable of finding good variational parameters $\tau$ and $t$. Now, sampling from this final state $\psi_{f}$ will return both MISs of the graph with high probability. Note that listing all maximal independent sets of a graph is also NP, and can be used as a subroutine for solving many NP-complete graph problems. 